# SETUP (MUST RUN EACH TIME)

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-09fcda22-e9ad-6552-57f0-5e9a1f215e6f)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Stylegan2-ada-pytorch Prerequisites

In [3]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [4]:
torch.cuda.device_count()

1

In [5]:
import torchvision

In [6]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 195 kB 12.7 MB/s 
     |████████████████████████████████| 108 kB 50.6 MB/s 
     |████████████████████████████████| 26.9 MB 1.3 MB/s 


## Get the StyleGAN code

In [ ]:
# WARNING : conv2d_gradfix not supported on PyTorch {torch.__version__}.
# !git clone https://github.com/derekphilipau/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 2.85 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [7]:
!git clone https://github.com/snoop2head/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134
Receiving objects: 100% (134/134), 1.12 MiB | 20.51 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [8]:
mkdir /content/stylegan2-ada-pytorch/datasets

In [9]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


# FIRST RUN (RUN ONLY ONCE)

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  Your training data will be stored in **MachineLearningForArtists/MyProject**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/Ghibli3

## Prepare Your Image Dataset for StyleGAN


### Unzip your dataset

***If your images are not zipped, skip this step.***

If your images are in a .zip file, you will need to unzip them.   Copy the path to your .zip file and paste here under `zip_path`. 


In [ ]:
zip_path = '/content/resize_square-20220208T134536Z-001.zip' #@param {type:"string"}

In [ ]:
!unzip "$zip_path"

### Prepare your dataset for StyleGAN


Copy the path to your unzipped dataset folder of images and paste into `dataset_path`

In [10]:
dataset_path = '/content/drive/MyDrive/resize_square' #@param {type:"string"}

The following commands will prepare your dataset.  It will crop all of your images to 512x512 pixels.  ***It will NOT overwrite your original images.***

In [11]:
!apt install imagemagick webp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [12]:
!mkdir /content/stylegan2-ada-pytorch/datasets/raw

In [13]:
!mkdir /content/stylegan2-ada-pytorch/datasets/processed

In [14]:
!cp "$dataset_path"/* /content/stylegan2-ada-pytorch/datasets/raw/

In [15]:
%cd /content/stylegan2-ada-pytorch/datasets/raw/

/content/stylegan2-ada-pytorch/datasets/raw


In [16]:
!mogrify -type TrueColor -set colorspace sRGB -colorspace sRGB -resize 512x512 -background white -gravity center -extent 512x512 -format jpg -path ../processed *

In [17]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [18]:
!python dataset_tool.py --source=/content/stylegan2-ada-pytorch/datasets/raw/ --dest=./datasets/stylegan_dataset.zip

100% 460/460 [00:03<00:00, 125.79it/s]


In [19]:
!cp ./datasets/stylegan_dataset.zip /content/drive/MyDrive/MachineLearningForArtists/Ghibli3/

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/Ghibli3 --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/stylegan_dataset.zip",
    "use_labels": false,
    "max_size": 460,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16

## Go to bed :)

Normal training of a dataset can take days.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/MyProject

Each training run is stored in a separate directory.  The initial training run is stored in `00000-stylegan_dataset...`.  If you run training twice, the second run will be stored in `00001-stylegan_dataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  In it you should see various images from your original dataset. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.


# RESUME TRAINING

***Make sure you run the above section "SETUP (MUST RUN EACH TIME)" before resuming training.***


In [ ]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


Copy previously saved datset

In [ ]:
!cp /content/drive/MyDrive/MachineLearningForArtists/Ghibli3/stylegan_dataset.zip ./datasets/

If you had to stop training, the browser window closed, or the Colab session timed out (12-24 hours), you can resume training by using a .pkl file stored on your Google Drive.  For your project, this file will be in `drive/MyDrive/MachineLearningForArtists/MyProject/0000X-stylegan_datset...`

In [20]:
#@title PKL File Path 
#@markdown Copy & Paste .pkl file path here:
pkl_file = '/content/drive/MyDrive/MachineLearningForArtists/Ghibli3/00000-stylegan_dataset-mirror-paper512/network-snapshot-000000.pkl' #@param {type:"string"}


Run the training, resuming with the last save .pkl file.  Results will be stored in your Google Drive under `MachineLearningForArtists/MyProject`

In [21]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [ ]:
!python train.py --resume=$pkl_file --outdir=/content/drive/MyDrive/MachineLearningForArtists/Ghibli3 --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/stylegan_dataset.zip",
    "use_labels": false,
    "max_size": 460,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16

# OPTIONAL: TRANSFER LEARNING

## OR Use Transfer Learning (Optional)

We don't have to train from scratch.  We can train using *transfer learning*.  We will use a StyleGAN2 model already pre-trained on faces to train a new model that is trained against our image dataset.

### Get the faces model

In [22]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


In [ ]:
!apt-get install megatools

다운로드 ffhq-512-avg-tpurun1.pkl 


In [24]:
!megadl https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng

Downloaded ffhq-512-avg-tpurun1.pkl


### Prepare old Faces model for StyleGAN2-ada-pytorch


source파일을 dest파일로 복사

In [25]:
!python legacy.py \
    --source=/content/stylegan2-ada-pytorch/ffhq-512-avg-tpurun1.pkl \
    --dest=stylegan2-ada-pytorch-ffhq-512.pkl

Loading "/content/stylegan2-ada-pytorch/ffhq-512-avg-tpurun1.pkl"...
Traceback (most recent call last):
  File "legacy.py", line 318, in <module>
    convert_network_pickle() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "legacy.py", line 309, in convert_network_pickle
    data = load_network_pkl(f, force_fp16=force_fp16)
  File "legacy.py", line 26, in load_network_pkl
    G = convert_tf_generator(tf_G)
  File "legacy.py", line 155, in convert_tf_generator
    raise ValueError('Unknown TensorFlow kwarg', un

### Train using the faces model with your dataset 

In [27]:
!python train.py --resume=/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch-ffhq-512.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/Ghibli3 --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/stylegan_dataset.zip",
    "use_labels": false,
    "max_size": 460,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16